For this demonstration we will be simulating a SrTiO3 CeO2 interface downloaded from https://www.materialscloud.org/explore/stoceriaitf/grid/calculations. We first import the multislice library, read in and plot the crystal and set up some basic parameters for our simulation.

In [ ]:
import pyms
import numpy as np
import matplotlib.pyplot as plt

# Probe accelerating voltage in eV
eV = 3e5

#No object forming aperture
app = None

# Grid shape
gridshape = [512,512]

thicknesses = 100

# Make SrTiO3/CeO2 interface structure
# Get crystal
crystal = pyms.structure(
    "Structures/SrTiO3_CeO2_interface.xyz", atomic_coordinates="cartesian"
)

# tiling 
tiling = [1,7]

# A few maniupulations to remove vaccuum at edges and create a psuedo-periodic
# structure
crystal = crystal.slice([0.1, 0.76], axis=0)
from copy import deepcopy

other = deepcopy(crystal).slice([0.017, 0.99], axis=0)
other.reflect([0])
crystal = crystal.concatenate(other, axis=0)

# Subslicing of crystal for multislice
subslices = [0.33, 0.66, 1.0]

# Output structure for examination in Vesta
crystal.output_vesta_xtl("manipulated.xtl")

# Grid size
gridsize = np.asarray(tiling)*crystal.unitcell[:2]

# Target the oxygen K edge
Ztarget = 8

#Defocii for an image series
defocii = np.arange(-300,301,150)

# principal and orbital angular momentum quantum numbers for bound state
n=1
ell=0
epsilon = 1

EFTEM_images = pyms.EFTEM(
    crystal,
    gridshape,
    eV,
    app,
    thicknesses,
    Ztarget,
    n,
    ell,
    epsilon,
    df=defocii,
    subslices=subslices,
    device_type=None,
    tiling=tiling,
    nT=5,
)

#Plot image series
fig,axes = plt.subplots(nrows= EFTEM_images.shape[0],figsize=(4,16))
for ax,img,df in zip(axes,EFTEM_images,defocii):

    ax.imshow(img)
    ax.set_axis_off()
    ax.set_title('Defocus {0:5.0f} \AA'.format(df),usetex=True)
